In [1]:
!pip install sacremoses
!pip install datasets
!clear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [2]:
from transformers import BertModel, BertTokenizer
import datasets
import torch
import pandas

In [3]:
# Tokenizer training
# https://ai.plainenglish.io/bert-pytorch-implementation-prepare-dataset-part-1-efd259113e5a
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased").to("cuda")

BATCH_SIZE = 32

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

csv = pandas.read_csv("TrainReviews.csv").sample(frac=1).reset_index(drop=True)
train_csv = csv[0:3000].reset_index(drop=True)
valid_csv = csv[3000:3870].reset_index(drop=True)

class ReviewDataset(torch.utils.data.Dataset):
  def __init__(self, pandas_array):
    tokenized = tokenizer.batch_encode_plus(
        pandas_array["review"],
        truncation=True,
        max_length=512,
        padding=True,
        return_tensors="pt",
    )

    self.tokenized = tokenized
    self.classes = pandas_array["class"]

  def __getitem__(self, idx):
    return self.tokenized["input_ids"][idx], self.tokenized["attention_mask"][idx], torch.tensor(self.classes[idx])

  def __len__(self):
    return len(self.classes)

train_dataset = ReviewDataset(train_csv)
valid_dataset = ReviewDataset(valid_csv)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
class Classifier(torch.nn.Module):
  def __init__(self, bert_model, r=16, alpha=16):
    super(Classifier, self).__init__()
    for param in bert_model.parameters():
      param.requires_grad = False
    self.bert_model = bert_model
    self.linear = torch.nn.Linear(768, 1)
    self.sigmoid = torch.nn.Sigmoid()

    # Initialize LoRA matrices
    self.lora_A = torch.nn.Parameter(torch.randn(768, r) / r**0.5)
    self.lora_B = torch.nn.Parameter(torch.randn(r, 1))
    self.scaling = alpha / r

  def forward(self, x, attention_mask):
    y = x.clone().detach()
    x = self.bert_model(x, attention_mask=attention_mask)

    # Apply LoRA to linear layer
    x_lora = self.linear(x.pooler_output) + (x.pooler_output @ self.lora_A @ self.lora_B) * self.scaling
    x = self.sigmoid(x_lora)

    return x

In [8]:
classifier = Classifier(model).to("cuda")
optimizer = torch.optim.AdamW(classifier.parameters(), lr=1e-3)
criterion = torch.nn.BCELoss()

num_epochs = 4
for epoch in range(num_epochs):
  classifier.train()
  for i, (input_ids, attention_mask, labels) in enumerate(train_dataloader):
    labels = labels.float().to("cuda").view(-1, 1)
    input_ids = input_ids.to("cuda")
    attention_mask = attention_mask.to("cuda")

    out = classifier(input_ids, attention_mask=attention_mask)

    optimizer.zero_grad()
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()

    if i%10 == 0:
      print(f"Epoch: {epoch+1}/{num_epochs}; Step: {i}/{len(train_dataloader)}; Loss: {loss}")

  with torch.no_grad():
    loss_sum = 0
    acc_sum = 0

    classifier.eval()
    for i, (input_ids, attention_mask, labels) in enumerate(valid_dataloader):
      labels = labels.float().to("cuda").view(-1, 1)
      input_ids = input_ids.to("cuda")
      attention_mask = attention_mask.to("cuda")

      out = classifier(input_ids, attention_mask=attention_mask)
      loss = criterion(out, labels)
      loss_sum += loss
      acc_sum += torch.sum(torch.round(out) == labels) / labels.shape[0]


    avg_loss = loss_sum/len(valid_dataloader)
    avg_acc = acc_sum/len(valid_dataloader)
    print(f"###### VALIDATION ###### Loss: {avg_loss}; Accuracy: {avg_acc}")


Epoch: 1/4; Step: 0/94; Loss: 1.6076180934906006
Epoch: 1/4; Step: 10/94; Loss: 0.9943917989730835
Epoch: 1/4; Step: 20/94; Loss: 0.9367997646331787
Epoch: 1/4; Step: 30/94; Loss: 2.038233757019043
Epoch: 1/4; Step: 40/94; Loss: 0.6408807039260864
Epoch: 1/4; Step: 50/94; Loss: 0.9567174315452576
Epoch: 1/4; Step: 60/94; Loss: 0.7989353537559509
Epoch: 1/4; Step: 70/94; Loss: 0.7938232421875
Epoch: 1/4; Step: 80/94; Loss: 0.5553081035614014
Epoch: 1/4; Step: 90/94; Loss: 0.4310173988342285
###### VALIDATION ###### Loss: 0.7370535135269165; Accuracy: 0.7295387387275696
Epoch: 2/4; Step: 0/94; Loss: 0.6964477300643921
Epoch: 2/4; Step: 10/94; Loss: 0.41688522696495056
Epoch: 2/4; Step: 20/94; Loss: 0.4156584143638611
Epoch: 2/4; Step: 30/94; Loss: 0.31409090757369995
Epoch: 2/4; Step: 40/94; Loss: 0.25206828117370605
Epoch: 2/4; Step: 50/94; Loss: 0.16730739176273346
Epoch: 2/4; Step: 60/94; Loss: 0.33915644884109497
Epoch: 2/4; Step: 70/94; Loss: 0.5338138937950134
Epoch: 2/4; Step: 80/